In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from keras.callbacks import ReduceLROnPlateau,EarlyStopping

In [2]:

batch_size = 100
img_height = 300
img_width = 300


In [3]:

train_set = tf.keras.preprocessing.image_dataset_from_directory(
   "C:\\Users\\sarthak mohapatra\\Downloads\\archive\\data\\train",
    image_size= (img_height, img_width),
    batch_size=batch_size

)

test_set = tf.keras.preprocessing.image_dataset_from_directory(
    "C:\\Users\\sarthak mohapatra\\Downloads\\archive\\data\\test",
    image_size= (img_height, img_width),
    batch_size=batch_size)

val_set =  tf.keras.preprocessing.image_dataset_from_directory(
 "C:\\Users\\sarthak mohapatra\\Downloads\\archive\\data\\val",

    image_size= (img_height, img_width),
    batch_size=batch_size)

Found 791 files belonging to 2 classes.
Found 100 files belonging to 2 classes.
Found 98 files belonging to 2 classes.


In [4]:
class_names = train_set.class_names

print(class_names)


['Accident', 'Non Accident']


In [5]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train = train_set.cache().prefetch(buffer_size=AUTOTUNE)
test = test_set.cache().prefetch(buffer_size=AUTOTUNE)

img_shape = (img_height, img_width, 3)

In [6]:
base_model = tf.keras.applications.MobileNetV2(input_shape=img_shape,include_top=False, weights='imagenet')

base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    layers.Conv2D(32, 3, activation='relu'),
    layers.Conv2D(64, 3, activation='relu'),
    layers.Conv2D(128, 3, activation='relu'),
    layers.Flatten(),
    layers.Dense(2, activation= 'softmax')
])

In [7]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])
lr=ReduceLROnPlateau(monitor='val_accuracy',patience=2,verbse=0.5,factor=0.5,min_lr=0.0001)
early_stop=EarlyStopping(monitor='val_loss',patience=2,restore_best_weights=True)


In [8]:
history = model.fit(train, validation_data = val_set, epochs = 20,callbacks=[lr])

Epoch 1/20


8/8 [==============================] - 31s 2s/step - loss: 0.8631 - accuracy: 0.5550 - val_loss: 0.5896 - val_accuracy: 0.7143 - lr: 0.0010
Epoch 2/20
8/8 [==============================] - 17s 2s/step - loss: 0.5413 - accuracy: 0.7459 - val_loss: 0.4275 - val_accuracy: 0.8163 - lr: 0.0010
Epoch 3/20
8/8 [==============================] - 17s 2s/step - loss: 0.3887 - accuracy: 0.8255 - val_loss: 0.3305 - val_accuracy: 0.8571 - lr: 0.0010
Epoch 4/20
8/8 [==============================] - 16s 2s/step - loss: 0.2278 - accuracy: 0.9128 - val_loss: 0.2961 - val_accuracy: 0.8980 - lr: 0.0010
Epoch 5/20
8/8 [==============================] - 17s 2s/step - loss: 0.1497 - accuracy: 0.9381 - val_loss: 0.2450 - val_accuracy: 0.8776 - lr: 0.0010
Epoch 6/20
8/8 [==============================] - 17s 2s/step - loss: 0.0741 - accuracy: 0.9785 - val_loss: 0.2246 - val_accuracy: 0.9184 - lr: 0.0010
Epoch 7/20
8/8 [==============================] - 18s 2s/step - loss: 0.0555 - accuracy: 0.9

In [9]:
def predict_frame(img):
    img_array = tf.keras.utils.img_to_array(img)
    img_batch = np.expand_dims(img_array, axis=0)
    prediction=(model.predict(img_batch) > 0.5).astype("int32")
    if(prediction[0][0]==0):
        return("Accident Detected")
    else:
        return("No Accident")

In [10]:
import cv2

In [11]:

# label=[]
# c=1
# cap=cv2.VideoCapture(0)
# #cap= cv2.VideoCapture("C:\\Users\\sarthak mohapatra\\Downloads\\car vs bike accident cctv video in pollachi.mp4")
# # if not cap.isOpened():
# #     print("Error: Could not open video file.")
# #     exit()
# # while True:
# #     grabbed, frame = cap.read()
  
# #     if not grabbed:
# #       print("Warning: Failed to read frame from video file.")  
# #       break
# #     if c%30==0:
# #     #  if frame is not None: 
             
# #           #resized_frame=tf.keras.preprocessing.image.smart_resize(frame, (300, 300), interpolation='bilinear')
# #         # image.append(frame)
# #      label.append(predict_frame(frame))
# #         # if(len(image)==75):
# #     cv2.imshow('frame',frame)    #     break
# #     c+=1
#     # cap.release()
# while True:
#     ret, frame = cap.read()
#     if c%30==0:
#      print(c)
#      #resize_frame=tf.keras.preprocessing.image.smart_resize(frame,(300,300),interpolation='bilinear')
#      predict=predict_frame(frame)
#      label.append(predict)
#      c+=1
#     cv2.imshow('frame',frame)
#     key=cv2.waitKey(1)&0xFF
#     if  key == ord("a"):
#         break
# cap.release()
# cv2.destroyAllWindows()



In [92]:
label=[]
c=1
cap=cv2.VideoCapture(r"C:\Users\sarthak mohapatra\Downloads\15 minutes of heavy traffic noise in India _ 14-08-2022.mp4")
while True:
    ret, frame = cap.read()
    if c%30==0:
     print(c)
    #ret, frame = cap.read()
     resize_frame=cv2.resize(frame,(300,300))
  
     predict=predict_frame(resize_frame)
     label.append(predict)
    c+=1
    cv2.imshow('frame',frame)
    key=cv2.waitKey(1)&0xFF
    if  key == ord("a"):
        break
cap.release()
cv2.destroyAllWindows()


30
1/1 [==============================] - 0s 79ms/step
60
1/1 [==============================] - 0s 74ms/step
90
1/1 [==============================] - 0s 81ms/step
120
1/1 [==============================] - 0s 72ms/step
150
1/1 [==============================] - 0s 75ms/step
180
1/1 [==============================] - 0s 68ms/step
210
1/1 [==============================] - 0s 66ms/step
240
1/1 [==============================] - 0s 68ms/step
270
1/1 [==============================] - 0s 84ms/step
300
1/1 [==============================] - 0s 65ms/step
330
1/1 [==============================] - 0s 79ms/step
360
1/1 [==============================] - 0s 65ms/step
390
1/1 [==============================] - 0s 65ms/step
420
1/1 [==============================] - 0s 59ms/step
450
1/1 [==============================] - 0s 67ms/step
480
1/1 [==============================] - 0s 64ms/step
510
1/1 [==============================] - 0s 64ms/step
540
1/1 [==============================] - 0s 61ms/

In [99]:
print(label[10])

No Accident
